In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)


train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split = 0.2,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.7,
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=200,
        #color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=50,
#         color_mode ='rgb',
        shuffle =True,
        seed = 42,
        class_mode='categorical')

Found 36000 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [2]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3,
                 activation='selu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, 3, activation='selu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, 3, activation='selu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
# model.add(Dense(8, activation='selu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=150,
    validation_data=test_generator,
    validation_steps=10)

model.save('model-16')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 39200)             0

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/150
10/10 [==============================] - 169s 17s/step - loss: 2.4216 - accuracy: 0.3785 - val_loss: 1.1894 - val_accuracy: 0.4280
Epoch 2/150
10/10 [==============================] - 186s 19s/step - loss: 0.9850 - accuracy: 0.5675 - val_loss: 0.9225 - val_accuracy: 0.5680
Epoch 3/150
10/10 [==============================] - 178s 18s/step - loss: 0.8837 - accuracy: 0.6060 - val_loss: 0.7984 - val_accuracy: 0.6640
Epoch 4/150
10/10 [==============================] - 174s 17s/step - loss: 0.8165 - accuracy: 0.6245 - val_loss: 0.8195 - val_accuracy: 0.6360
Epoch 5/150
10/10 [==============================] - 182s 18s/step - loss: 0.7813 - accuracy: 0.6630 - val_loss: 0.7219 - val_accuracy: 0.7020
Epoch 6/150
10/10 [==============================] - 165s 16s/step - loss: 0.7655 - accuracy: 0.6560 - val_loss: 0.7129 - val_accuracy: 0.6960
Epoch 7/150
10/10 [==============================] - 173s 17s/step - loss: 0.7719 - accuracy: 0.6660 - val_loss: 0.7146 - val_accuracy: 0.7140

Epoch 58/150
10/10 [==============================] - 160s 16s/step - loss: 0.4830 - accuracy: 0.7980 - val_loss: 0.4616 - val_accuracy: 0.8200
Epoch 59/150
10/10 [==============================] - 165s 16s/step - loss: 0.5277 - accuracy: 0.7900 - val_loss: 0.5009 - val_accuracy: 0.8000
Epoch 60/150
10/10 [==============================] - 163s 16s/step - loss: 0.5058 - accuracy: 0.7890 - val_loss: 0.5541 - val_accuracy: 0.7660
Epoch 61/150
10/10 [==============================] - 167s 17s/step - loss: 0.4904 - accuracy: 0.8025 - val_loss: 0.5927 - val_accuracy: 0.7380
Epoch 62/150
10/10 [==============================] - 163s 16s/step - loss: 0.5107 - accuracy: 0.7995 - val_loss: 0.4931 - val_accuracy: 0.8020
Epoch 63/150
10/10 [==============================] - 164s 16s/step - loss: 0.5201 - accuracy: 0.7815 - val_loss: 0.4765 - val_accuracy: 0.8200
Epoch 64/150
10/10 [==============================] - 162s 16s/step - loss: 0.4910 - accuracy: 0.8070 - val_loss: 0.4664 - val_accuracy:

Epoch 115/150
10/10 [==============================] - 166s 17s/step - loss: 0.4870 - accuracy: 0.7955 - val_loss: 0.4261 - val_accuracy: 0.8540
Epoch 116/150
10/10 [==============================] - 169s 17s/step - loss: 0.5054 - accuracy: 0.7965 - val_loss: 0.4929 - val_accuracy: 0.8020
Epoch 117/150
10/10 [==============================] - 162s 16s/step - loss: 0.4994 - accuracy: 0.8010 - val_loss: 0.4332 - val_accuracy: 0.8440
Epoch 118/150
10/10 [==============================] - 163s 16s/step - loss: 0.4360 - accuracy: 0.8220 - val_loss: 0.5243 - val_accuracy: 0.8060
Epoch 119/150
10/10 [==============================] - 176s 18s/step - loss: 0.4747 - accuracy: 0.8070 - val_loss: 0.4130 - val_accuracy: 0.8420
Epoch 120/150
10/10 [==============================] - 168s 17s/step - loss: 0.4602 - accuracy: 0.8255 - val_loss: 0.4248 - val_accuracy: 0.8080
Epoch 121/150
10/10 [==============================] - 165s 16s/step - loss: 0.4789 - accuracy: 0.8180 - val_loss: 0.4981 - val_ac